In [11]:
import ipywidgets as widgets
import json
import matplotlib.image as img
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import clear_output

In [12]:
# Game Info
with open("../data/2017/playoffs.json") as file:
    data = json.load(file)

# use the first game to debug
debug_game = data[0]

plays = debug_game['plays']
plays_info = []
for play in plays:
    details = play.get('details', {})
    periodDescriptor = play.get('periodDescriptor', {})
    row = {
        'typeDescKey': play.get('typeDescKey'),
        'timeInPeriod': play.get('timeInPeriod'),
        'timeRemaining': play.get('timeRemaining'),
        'xCoord': details.get('xCoord'),
        'yCoord': details.get('yCoord'),
        'period': periodDescriptor.get('number')
    }
    plays_info.append(row)

df = pd.DataFrame(plays_info)


In [13]:
# Initialize the widgets
plot_widget = widgets.Output()
slider_widget = widgets.IntSlider(value=0, min=0, max=df.shape[0]-1, step=1, description='Play #')
combined_widget = widgets.VBox(
    [plot_widget, slider_widget],
    layout=widgets.Layout(
        justify_content='center', 
        align_items='center'
    ))

In [14]:
def init_plot(ax):
    with plot_widget:
        x_min = -100
        x_max = 100
        y_min = -42.5
        y_max = 42.5
        rink_img = img.imread("../../figures/nhl_rink.png")

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        
        ax.imshow(rink_img, extent=(x_min, x_max, y_min, y_max))

        ax.set_xlabel('X Coordinate (ft)')
        ax.set_ylabel('Y Coordinate (ft)')

def plot_play(play_number):
    with plot_widget:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        
        init_plot(ax)
        play = df.iloc[play_number]
        
        ax.set_title(f"Game Play-By-Play")
        x = play['xCoord']
        y = play['yCoord']

        if pd.notna(x) and pd.notna(y):
            ax.plot(x, y, 'bo')

        plt.show()
        print(f"Event: {play['typeDescKey']}")
        print(f"Time in period: {play['timeInPeriod']}")
        print(f"Period: {play['period']}")

In [15]:
slider_widget.observe(lambda change: plot_play(change['new']), names='value')

display(combined_widget)
plot_play(slider_widget.value)